# Test Code for the Galactic Plane Filter Metric

Code adapted from an example by Lynne Jones

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import rubin_sim.maf as maf
from rubin_sim.data import get_data_dir

Load the baseline v2.0 as a test case OpSim:

In [2]:
from rubin_sim.data import get_baseline

opsim_fname = get_baseline()
print(opsim_fname)

runName = os.path.split(opsim_fname)[-1].replace('.db', '')
print(runName)
opsim_db = maf.OpsimDatabase(opsim_fname)

/Users/rstreet1/rubin_sim_data/sim_baseline/baseline_v2.0_10yrs.db
baseline_v2.0_10yrs


Taking an example sky location within the Galactic Bulge:

In [3]:
test_ra = (17.0 + 57.0/60.0 + 34.0/3600.0)*15.0
test_dec = (29.0 + 13.0/60.0 + 15.0/3600.0)*-1.0
test_slicer = maf.UserPointsSlicer(test_ra, test_dec)

In [4]:
def load_map_data(file_path):

    with fits.open(file_path) as hdul:
        map_data_table = hdul[1].data

    return map_data_table

In [5]:
import numpy as np
import healpy as hp
import rubin_sim.maf as maf
from astropy import units as u
from astropy_healpix import HEALPix
from astropy.coordinates import Galactic, TETE, SkyCoord
from astropy.io import fits

class galPlaneTimePerFilter(maf.BaseMetric):
    """Metric to evaluate for each HEALpix, the fraction of exposure time spent in each filter as
     a fraction of the total exposure time dedicated to that HEALpix.  The metric sums this over
     all HEALpix in the Galactic Plane/Magellanic Clouds region of interest and all filters and
     presents the result as a fraction of the value expected from the optimal survey strategy.

    Parameters
    ----------
    fieldRA : float, RA in degrees of a given pointing
    fieldDec : float, Dec in degrees of a given pointing
    filter : str, filter bandpass used for a given observation
    observationStartMJD : float, MJD timestamp of the start of a given observation
    visitExposureTime : float, exposure time in seconds
    """

    def __init__(
        self,
        cols=[
            "fieldRA",
            "fieldDec",
            "filter",
            "observationStartMJD",
            "visitExposureTime",
            "fiveSigmaDepth",
        ],
        metricName="galPlaneTimePerFilter",
        **kwargs
    ):
        """Kwargs must contain:
        filters  list Filterset over which to compute the metric
        """

        self.ra_col = "fieldRA"
        self.dec_col = "fieldDec"
        self.filterCol = "filter"
        self.m5Col = "fiveSigmaDepth"
        self.mjdCol = "observationStartMJD"
        self.exptCol = "visitExposureTime"
        self.filters = ["u", "g", "r", "i", "z", "y"]
        self.magCuts = {
            "u": 22.7,
            "g": 24.1,
            "r": 23.7,
            "i": 23.1,
            "z": 22.2,
            "y": 21.4,
        }
        cwd = os.getcwd()
        self.MAP_DIR = get_data_dir()
        self.MAP_FILE_ROOT_NAME = "priority_GalPlane_footprint_map_data"
        self.load_maps()
        self.calc_coaddmap()

        super().__init__(col=cols, metricName=metricName, metricDtype="object")

    def load_maps(self):
        self.NSIDE = 64
        self.NPIX = hp.nside2npix(self.NSIDE)
        for f in self.filters:
            file_path = os.path.join(
                self.MAP_DIR,
                "maf",
                self.MAP_FILE_ROOT_NAME + "_" + str(f) + ".fits",
                )
            map_data_table = load_map_data(file_path)

            setattr(self, "map_" + str(f), map_data_table['combined_map'])
            setattr(self, "map_data_" + str(f), map_data_table)

    def calc_coaddmap(self):
        """For each HEALpix in the sky, sum the value of the priority weighting
        per filter.  This gives us the normalization factor to calculate the
        fractional proportional of exposure time"""

        self.coadded_maps = {}
    
        for i, f in enumerate(self.filters):
            # Fetch the map data for this filter
            map_data_table = getattr(self, 'map_data_'+str(f))
            
            for col in map_data_table.columns:
                if col.name in self.coadded_maps.keys():
                    coadded_map = self.coadded_maps[col.name]
                else:
                    coadded_map = np.zeros(self.NPIX)
            
                # Extract the HEALpix priority data for the current science case
                fmap = map_data_table[col.name]
                
                # Sum the priority values for each HEALpix in this science region
                # over all filters
                coadded_map += fmap
                
                self.coadded_maps[col.name] = coadded_map

    def calc_idealfExpT(self, filter_name, pixels, col):
        """Method to calculate the optimal value of the fExpT metric, in the event
        that a survey strategy perfectly overlaps the desired Galactic Plane footprint
        and the exposure time spent in the different filters at the desired relative
        proportions.  Calculation is made over all the pixels in a given dataSlice,
        so that the metric can be accurately normalized later."""

        fexpt_per_pixel = np.zeros(len(pixels))

        # Extract the HEALpix priority data for the current science case in
        # the appropriate filter.  This returns a list of priority values for 
        # all HEALpixels, with zero values for regions outside the area of interest
        map_data_table = getattr(self, 'map_data_'+str(filter_name))
        fmap = map_data_table[col.name]

        # First select the pixels within the science region of interest for the
        # current filter.  The survey region for a given science case can vary
        # between filters
        idx = np.where(fmap > 0.0)[0]
        
        # The ideal ratio of observations in this filter, relative to all observations
        # for a given pixel is estimated from the ratio of priority per pixel to 
        # the priority of each pixel summed over all filters.  
        idealfExpT = fmap[idx].sum() / self.coadded_maps[col.name][idx].sum()
        
        return idealfExpT

    def calculate_overlap_survey_region(self,region_pixels, dataSlice, match):
        
        # Calculate which HEALpixels observations in the dataSlice correspond to
        coords_icrs = SkyCoord(
            dataSlice[self.ra_col][match],
            dataSlice[self.dec_col][match],
            frame="icrs",
            unit=(u.deg, u.deg),
        )
        coords_gal = coords_icrs.transform_to(Galactic())
        ahp = HEALPix(nside=self.NSIDE, order="ring", frame=TETE())
        pixels = ahp.skycoord_to_healpix(coords_gal)
        
        # Calculate the overlap between the observed HEALpixels and
        # those from the desired survey region
        overlap_pixels = list(set(pixels.tolist()).intersection(set(region_pixels.tolist())))
        
        # Identify which observations from the dataSlice correspond to 
        # the overlapping survey region.  This may produce multiple
        # indices in the array, referred to different observations
        match = np.array(match)
        match_obs = []
        for p in overlap_pixels:
            ip = np.where(pixels == p)[0]
            match_obs += match[ip].tolist()
        
        return overlap_pixels, match_obs

    def run(self, dataSlice, slicePoint=None):

        # Pre-calculating data that will be used later
        total_expt_per_pixel = dataSlice[self.exptCol].sum()
        
        # This metric is calculated both for the combined priority map as well
        # as the maps for the component science cases, hence the returned 
        # metric_data is a dictionary.  ideal_data contains the corresponding
        # metric values if the survey strategy returned ideal results for 
        # each given science case; these data are used for normalization
        metric_data = {}
         
        for i, f in enumerate(self.filters):
            # Fetch the map data for this filter
            map_data_table = getattr(self, 'map_data_'+str(f))
            
            # Select observations within the OpSim for the current filter
            # which match the S/N requirement, and extract the exposure times
            # for those observations
            idx1 = np.where(dataSlice[self.filterCol] == f)[0]
            idx2 = np.where(dataSlice[self.m5Col] >= self.magCuts[f])[0]
            match = list(set(idx1).intersection(set(idx2)))
            
            # Calculate which HEALpixels these observations correspond to
            coords_icrs = SkyCoord(
                dataSlice[self.ra_col][match],
                dataSlice[self.dec_col][match],
                frame="icrs",
                unit=(u.deg, u.deg),
            )
            coords_gal = coords_icrs.transform_to(Galactic())
            ahp = HEALPix(nside=self.NSIDE, order="ring", frame=TETE())
            pixels = ahp.skycoord_to_healpix(coords_gal)

            for col in map_data_table.columns:
                    
                # Pixels within the current region of scientific interest:
                region_pixels = np.where(map_data_table[col.name] > 0.0)[0]
                
                # Now find the overlap between the HEALpix covered by the dataSlice
                # and the ideal survey region for this science case, and the 
                # dataSlice observations corresponding to this region
                (overlap_pixels, match_obs) = self.calculate_overlap_survey_region(region_pixels, dataSlice, match)
                print('N overlapping pixels ',len(overlap_pixels))
                print('N matching obs ',len(match_obs))
                
                # Calculate the expected fraction of the total exposure time
                # that would be dedicated to observations in this filter
                # summed over all HEALpixels in the desired region, if the 
                # survey strategy exactly matched the needs of the current science case
                idealfExpT = self.calc_idealfExpT(f, overlap_pixels, col)
                print('ideal f/ExpT = ',idealfExpT)
                
                # Now calculate the actual fraction of exposure time spend
                # in this filter summed over all HEALpix from the overlap region. 
                # If no exposures are expected in this filter, this returns 1 
                # on the principle that 100% of the expected observations are 
                # provided, and additional data in other filters is usually welcome
                print('Exposure time for matching exposures = ',dataSlice[self.exptCol][match_obs].sum())
                print('Total exposure time per pixel = ',total_expt_per_pixel)
                fexpt = dataSlice[self.exptCol][match_obs].sum() / total_expt_per_pixel
                print('Exp fraction = ',fexpt)
                if idealfExpT > 0:
                    metric = fexpt / idealfExpT
                else:
                    metric = 1.0
                print('Exp fraction relative to ideal fraction = ',metric,f,col.name)
                
                # Accumulate the product of this metric over all filters for each science region
                if col.name not in metric_data.keys():
                    metric_data[col.name] = metric
                else:
                    metric_data[col.name] *= metric

        return metric_data


In [6]:
mymetric = galPlaneTimePerFilter()
sqlconstraint = None
bundle = maf.MetricBundle(mymetric, test_slicer, sqlconstraint, runName=runName)
g = maf.MetricBundleGroup({'test_metric': bundle}, opsim_db, outDir='test', resultsDb=None)

Calculate the metric:

In [7]:
g.runAll()

Querying database None with no constraint for columns ['fieldDec', 'filter', 'rotSkyPos', 'visitExposureTime', 'fieldRA', 'observationStartMJD', 'fiveSigmaDepth'].
Found 2086980 visits
Running:  ['test_metric']
N overlapping pixels  13
N matching obs  32
ideal f/ExpT =  0.07320802488598549
Exposure time for matching exposures =  960.0
Total exposure time per pixel =  24810.0
Exp fraction =  0.03869407496977025
Exp fraction relative to ideal fraction =  0.5285496368742714 u combined_map
N overlapping pixels  13
N matching obs  32
ideal f/ExpT =  0.05000000099346057
Exposure time for matching exposures =  960.0
Total exposure time per pixel =  24810.0
Exp fraction =  0.03869407496977025
Exp fraction relative to ideal fraction =  0.7738814840189903 u galactic_plane_map
N overlapping pixels  0
N matching obs  0
ideal f/ExpT =  nan
Exposure time for matching exposures =  0.0
Total exposure time per pixel =  24810.0
Exp fraction =  0.0
Exp fraction relative to ideal fraction =  1.0 u magelle

/var/folders/4z/rk7n9jss74g3shm9z6msmyvh0000gp/T/ipykernel_5630/1906731094.py:127: RuntimeWarning: invalid value encountered in true_divide
  idealfExpT = fmap[idx].sum() / self.coadded_maps[col.name][idx].sum()


N overlapping pixels  17
N matching obs  138
ideal f/ExpT =  0.2137683757409201
Exposure time for matching exposures =  4140.0
Total exposure time per pixel =  24810.0
Exp fraction =  0.16686819830713423
Exp fraction relative to ideal fraction =  0.7806028264413288 r combined_map
N overlapping pixels  17
N matching obs  138
ideal f/ExpT =  0.22500001127471014
Exposure time for matching exposures =  4140.0
Total exposure time per pixel =  24810.0
Exp fraction =  0.16686819830713423
Exp fraction relative to ideal fraction =  0.741636399757328 r galactic_plane_map
N overlapping pixels  0
N matching obs  0
ideal f/ExpT =  0.22222221891085311
Exposure time for matching exposures =  0.0
Total exposure time per pixel =  24810.0
Exp fraction =  0.0
Exp fraction relative to ideal fraction =  0.0 r magellenic_clouds_map
N overlapping pixels  7
N matching obs  66
ideal f/ExpT =  0.23076922876085604
Exposure time for matching exposures =  1980.0
Total exposure time per pixel =  24810.0
Exp fractio

N overlapping pixels  8
N matching obs  27
ideal f/ExpT =  0.1666666737821263
Exposure time for matching exposures =  810.0
Total exposure time per pixel =  24810.0
Exp fraction =  0.032648125755743655
Exp fraction relative to ideal fraction =  0.19588874617143115 y open_clusters_map
N overlapping pixels  0
N matching obs  0
ideal f/ExpT =  0.16666669260794148
Exposure time for matching exposures =  0.0
Total exposure time per pixel =  24810.0
Exp fraction =  0.0
Exp fraction relative to ideal fraction =  0.0 y zucker_sfr_map
N overlapping pixels  11
N matching obs  38
ideal f/ExpT =  0.1000000172323242
Exposure time for matching exposures =  1140.0
Total exposure time per pixel =  24810.0
Exp fraction =  0.045949214026602174
Exp fraction relative to ideal fraction =  0.4594920610848601 y pencilbeams_map
N overlapping pixels  13
N matching obs  42
ideal f/ExpT =  0.05877117063871482
Exposure time for matching exposures =  1260.0
Total exposure time per pixel =  24810.0
Exp fraction =  

In [8]:
bundle.metricValues

masked_array(data=[{'combined_map': 0.10427921014866028, 'galactic_plane_map': 0.18112338892014204, 'magellenic_clouds_map': 0.0, 'galactic_bulge_map': 0.000651943008137961, 'clementini_stellarpops_map': 0.0, 'bonito_sfr_map': 0.0, 'globular_clusters_map': 4.1151833025908737e-07, 'open_clusters_map': 0.0026247622403096794, 'zucker_sfr_map': 0.0, 'pencilbeams_map': 0.04030344334509967, 'xrb_priority_map': 0.5564694403701814}],
             mask=[False],
       fill_value=-666,
            dtype=object)